In [4]:
import pandas as pd 
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException
import time 
import re 
from bs4 import BeautifulSoup 
from tqdm import tqdm 

In [108]:
# css_selector 체크 함수 
def check_exists_by_css(driver, text):
    try:
        driver.find_element_by_css_selector(text)
    except NoSuchElementException:
        return False
    return True

### 서울 지역별 맛집 이름 가져오기

In [2]:
driver = webdriver.Chrome('/Users/sol/Downloads/chromedriver')

keyword = ['강서구 맛집', '양천구 맛집', '구로구 맛집', '금천구 맛집', '영등포구 맛집', '동작구 맛집', '관악구 맛집', '서초구 맛집', '강남구 맛집', '송파구 맛집', '강동구 맛집',
          '마포구 맛집', '용산구 맛집', '성동구 맛집', '광진구 맛집', '서대문구 맛집', '중구 맛집', '동대문구 맛집', '중랑구 맛집', '성북구 맛집', '은평구 맛집', '강북구 맛집', '도붕구 맛집', '노원구 맛집']

def scrapping(keyword):
    word_list = []
    df = pd.DataFrame()
    naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place'
    driver.get(naver_map_search_url)
    time.sleep(2)
    driver.switch_to.frame('searchIframe')
    time.sleep(2)

    for i in range(1,7):
        click = driver.find_element_by_xpath(f'//*[@id="app-root"]/div/div[2]/div[2]/a[{i}]')
        click.click()

        scroll_div = driver.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[1]")
        driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
        time.sleep(2)
        driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
        time.sleep(2)
        driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
        time.sleep(2)
        driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
        time.sleep(2)
        driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
        time.sleep(2)

        words = driver.find_elements_by_class_name('OXiLu')
        word = [word.text for word in words]
        word_list += word
            
    df['title'] = word_list
    df['keyword'] = keyword

    return df 

In [ ]:
full = pd.DataFrame()

for i in range(len(keyword)):
    test = scrapping(keyword[i])
    full = pd.concat([full, test], axis=0)

### 맛집 이름으로 url 가져오기

In [ ]:
full = full.drop_duplicates()
full = full.reset_index(drop=True)

full['naver_map_url'] = ''
title = full['keyword'] + full['title']
title[3397] = '성동구 맛집봉순이네다락방'

In [ ]:
for i, keyword in enumerate(title.tolist()): 
    print("이번에 찾을 키워드 :", i, f"/ {title.shape[0]} 행", keyword) 
    try: 
        naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place' # 검색 url 만들기 
        driver.get(naver_map_search_url) # 검색 url 접속, 즉 검색하기 
        time.sleep(3)
        driver.switch_to.frame('searchIframe')
        time.sleep(3) 
        
        try:
            click = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/ul/li[1]/div[2]/a[1]/div/div/span[1]')
            click.click()
        except:
            pass
        
        cu = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지 
        res_code = re.findall(r"place/(\d+)", cu) 
        final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#' 
        print(final_url) 
        full['naver_map_url'][i]=final_url 
        
    except IndexError: 
        full['naver_map_url'][i]= '' 
        print('none') 
#         test.to_csv('url_completed.csv', encoding = 'utf-8-sig')

### 가져온 url로 식당이름, 주소, 카테고리 가져오기 (이름과 주소는 크로스 체크를 위해서)

In [9]:
driver = webdriver.Chrome('/Users/sol/Downloads/chromedriver')

In [5]:
full = pd.read_csv('full.csv')

In [7]:
url = full['naver_map_url'].apply(lambda x:x.split('review')[0])
full['naver_home_url'] = url.apply(lambda x:x+'home')

In [ ]:
name_list = []
address_list = []
restaurant_classificaton_list = []

for i in range(len(full['naver_home_url'].tolist())):
    first_url = full['naver_home_url'][i]
    driver.get(first_url) 
    time.sleep(1)
    name = driver.find_elements_by_class_name('_3XamX')[0].text
    time.sleep(1)
    address = driver.find_elements_by_class_name('_2yqUQ')[0].text
    time.sleep(1)
    restaurant_classificaton= driver.find_elements_by_class_name('_3ocDE')[0].text
    time.sleep(1)
    
    name_list.append(name)
    address_list.append(address)
    restaurant_classificaton_list.append(restaurant_classificaton)  

In [ ]:
# full['new_title'] = name_list
test['new_title'] = name_list
# full['address'] = address_list
test['address'] = address_list
# full['category'] = restaurant_classificaton_list
test['category'] = restaurant_classificaton_list

### 평점, 아이디, 리뷰 가져오기

In [ ]:
# 수집할 정보들 
rating_list = [] # 평점 
user_id_list = [] # 유저의 id - 추천시스템에서는 필수 

for i in range(len(test['naver_map_url'])):
    
    url = full['naver_map_url'][i]
    
    driver.get(url) 
    time.sleep(2)
    driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._1fQmI > h2 > a').click()

    while True:  
        driver.find_element_by_tag_name('body').send_keys(Keys.END) 
        time.sleep(3) 
        if check_exists_by_css(driver,'#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._1fQmI > div._2kAri > a'):
            element = driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._1fQmI > div._2kAri > a')
            element.click() 
            time.sleep(3) 

        else:
            break

    time.sleep(1)

    # 식당 구분
    restaurant_name = test['title'][i]
    print('식당 이름 : '+restaurant_name) 

    avg = driver.find_element_by_css_selector("span[class='_1fvo3 Sv1wj']")
    avg_rate = avg.text
    print (avg_rate)

    div = driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._1fQmI > div')
    li = div.find_elements_by_class_name('_3FaRE')

    user_ids = div.find_elements_by_class_name('_16RxQ')
    user_id_list = [user_id.text for user_id in user_ids]

    for i in li:
        if check_exists_by_css(i,"span[class='_1fvo3 Sv1wj']"):
            rating = i.find_element_by_css_selector("span[class='_1fvo3 Sv1wj']")
            rate = rating.text
            rating_list.append(rate)
        else:
            rating_list.append('')
    
    output = {}

    output['name'] = restaurant_name
    output['user_id'] = user_id_list

    output_df = pd.DataFrame(output)
    
    test = pd.merge(test, output_df, how='outer', left_on='title', right_on='name')


식당 이름 : 오늘와인한잔 양천향교역점
4.8
식당 이름 : 오늘와인한잔 양천향교역점
4.34


In [1]:
test

NameError: name 'test' is not defined

In [ ]:
test

In [223]:
driver.quit()

In [226]:
# 수집할 정보들 
rating_list = [] # 평점 
user_id_list = [] # 유저의 id - 추천시스템에서는 필수 


driver.get(url) 
time.sleep(2)
driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._1fQmI > h2 > a').click()

while True:  
    driver.find_element_by_tag_name('body').send_keys(Keys.END) 
    time.sleep(3) 
    if check_exists_by_css(driver,'#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._1fQmI > div._2kAri > a'):
        element = driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._1fQmI > div._2kAri > a')
        element.click() 
        time.sleep(3) 
        
    else:
        break
        
time.sleep(1)
 
# 식당 구분
restaurant_name = test['title'][1]
print('식당 이름 : '+restaurant_name) 

avg = driver.find_element_by_css_selector("span[class='_1fvo3 Sv1wj']")
avg_rate = avg.text
print (avg_rate)

div = driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._1fQmI > div')
li = div.find_elements_by_class_name('_3FaRE')

user_ids = div.find_elements_by_class_name('_16RxQ')
user_id_list = [user_id.text for user_id in user_ids]




for i in li:
    if check_exists_by_css(i,"span[class='_1fvo3 Sv1wj']"):
        rating = i.find_element_by_css_selector("span[class='_1fvo3 Sv1wj']")
        rate = rating.text
        rating_list.append(rate)
    else:
        rating_list.append('')



#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._1fQmI > div > ul > li:nth-child(97) > div._3-LAD > span._1fvo3.Sv1wj > em

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._1fQmI > h2 > a"}
  (Session info: chrome=101.0.4951.54)
